# **Procesamiento de Lenguaje Natural**

## Maestría en Inteligencia Artificial Aplicada
#### Tecnológico de Monterrey
#### Prof Luis Eduardo Falcón Morales

### **Adtividad en Equipos:**

* **Nombres y matrículas:**

* 🧑‍💻 Ovidio Alejandro Hernández Ruano (A01796714)
* 🧑‍💻 José Manuel Toral Cruz (A01122243)
* 🧑‍💻 Oscar Enrique García García (A01016093)
* 🧑‍💻 Luis Gerardo Sanchez Salazar (A01232963)

* **Número de Equipo:**
* Equipo #20


# ¿Por qué elegimos la Propuesta 2?

Se recomienda avanzar con la Propuesta 2: sistema de explicabilidad sobre decisiones crediticias en RappiCard, debido a que presenta una combinación ideal de viabilidad técnica, alto impacto operativo y alineación regulatoria.

A diferencia de otras propuestas que requieren un desarrollo desde cero o fuerte dependencia de infraestructura externa, esta iniciativa parte de un modelo de riesgo ya validado y operativo en producción (Random Forest con SHAP), lo que la convierte en un subproducto natural de un sistema existente.

Además, su implementación es rápidamente ejecutable: la arquitectura está claramente definida, los datos ya están disponibles y el enfoque aprovecha técnicas estándar ampliamente aceptadas por reguladores (SHAP + LLM).

Más aún, el reto principal no es técnico, sino de compliance, diseño y validación de lenguaje explicativo (CX, legal), lo cual reduce la incertidumbre tecnológica y pone el foco en asegurar una comunicación clara, empática y conforme a la regulación.

Finalmente, la solución es altamente escalable y reutilizable: puede extenderse a otros productos, cohortes o canales (web, email, push), y abre la puerta a una capa de interpretabilidad auditada que fortalece la confianza del cliente y la transparencia del negocio.

A continuación, presentamos un MVP con datos simulados.

## Bibliotecas

In [ ]:
!pip install -q streamlit pandas torch accelerate transformers einops sentencepiece
# (opcional para cuantización GPTQ)
!pip install -q auto-gptq optimum
!pip install streamlit

In [ ]:
import streamlit as st
import pandas as pd
import openai
import textwrap
import json
from datetime import datetime
import pytz
import numpy as np
import random


## Datos simulados

Para poner en práctica esta idea, vamos a simular datos y features.


In [ ]:

from datetime import datetime, timedelta, timezone

# 1. Diccionario de variables con descripciones
variables_desc = {
    "BC_SCORE": "Calificación Buró de Crédito",
    "UTIL_LINEA_TDC": "Utilización de la línea de TDC",
    "SALDO_VIGENTE": "Saldo vigente en productos activos",
    "ANTIG_EDO_CTA": "Antigüedad del estado de cuenta (meses)",
    "TIEMPO_EMPLEO": "Tiempo en empleo actual (meses)",
    "INGRESOS_MENSUALES": "Ingresos mensuales declarados"
}
var_names = list(variables_desc.keys())

# 2. Simulación de una fila
def simulate_row(uid: int, seed: int = None):
    if seed is not None:
        random.seed(seed); np.random.seed(seed)

    raw = {
        "BC_SCORE": int(np.random.normal(700, 50)),
        "UTIL_LINEA_TDC": round(np.random.uniform(0, 1), 2),
        "SALDO_VIGENTE": round(np.random.uniform(0, 200_000), 2),
        "ANTIG_EDO_CTA": int(np.random.exponential(12)),
        "TIEMPO_EMPLEO": int(np.random.normal(36, 15)),
        "INGRESOS_MENSUALES": round(np.random.uniform(8_000, 80_000), 2)
    }

    shap = {
        k: round(np.random.normal(0, 0.15), 6)
        for k in var_names
    }

    beta = np.array([0.4, 0.2, 0.15, -0.1, -0.05, -0.2])
    x = np.array([
        raw["BC_SCORE"] / 850,
        raw["UTIL_LINEA_TDC"],
        raw["SALDO_VIGENTE"] / 250_000,
        raw["ANTIG_EDO_CTA"] / 60,
        raw["TIEMPO_EMPLEO"] / 120,
        raw["INGRESOS_MENSUALES"] / 100_000
    ])
    logit = x @ beta + np.random.normal(0, 0.25)
    score = 1 / (1 + np.exp(-logit))

    calc_date = datetime.now().replace(microsecond=0) - timedelta(minutes=random.randint(0, 3_600))
    req_date  = calc_date.replace(tzinfo=timezone.utc)

    return {
        "RPP_USER_ID": f"USR-{uid:05d}",
        "CALCULATION_DATE": calc_date,
        "FECHA_SOLICITUD": req_date,
        "SCORE": round(score, 3),
        "SHAP_JSON": json.dumps(shap),
        "VAR_JSON": json.dumps(raw)
    }

# 3. Generar DataFrame
def simulate_dataframe(n=200, seed=42):
    rows = [simulate_row(i, seed+i) for i in range(1, n+1)]
    df = pd.DataFrame(rows)
    return df.sort_values("CALCULATION_DATE").reset_index(drop=True)

# 4. Guardar datos
df_data = simulate_dataframe()
df_data.to_csv("snowflake_multiple_simulated.csv", index=False)

df_def = pd.DataFrame({
    "VARIABLE": var_names,
    "DESCRIPTION": [variables_desc[v] for v in var_names]
})
df_def.to_csv("definiciones_sim.csv", index=False, encoding="ISO-8859-1")

print("Datos simulados y definiciones guardados.")


Datos simulados y definiciones guardados.


In [ ]:
df_data

,RPP_USER_ID,CALCULATION_DATE,FECHA_SOLICITUD,SCORE,SHAP_JSON,VAR_JSON
0,USR-00063,2025-06-23 13:49:15,2025-06-23 13:49:15+00:00,0.707,"{""BC_SCORE"": 0.150837, ""UTIL_LINEA_TDC"": -0.01...","{""BC_SCORE"": 687, ""UTIL_LINEA_TDC"": 0.86, ""SAL..."
1,USR-00103,2025-06-23 14:20:15,2025-06-23 14:20:15+00:00,0.700,"{""BC_SCORE"": -0.006809, ""UTIL_LINEA_TDC"": 0.28...","{""BC_SCORE"": 658, ""UTIL_LINEA_TDC"": 0.65, ""SAL..."
2,USR-00198,2025-06-23 14:47:15,2025-06-23 14:47:15+00:00,0.680,"{""BC_SCORE"": -0.029789, ""UTIL_LINEA_TDC"": 0.16...","{""BC_SCORE"": 670, ""UTIL_LINEA_TDC"": 0.78, ""SAL..."
3,USR-00032,2025-06-23 15:04:15,2025-06-23 15:04:15+00:00,0.582,"{""BC_SCORE"": 0.143048, ""UTIL_LINEA_TDC"": 0.189...","{""BC_SCORE"": 730, ""UTIL_LINEA_TDC"": 0.86, ""SAL..."
4,USR-00174,2025-06-23 15:08:15,2025-06-23 15:08:15+00:00,0.634,"{""BC_SCORE"": -0.03313, ""UTIL_LINEA_TDC"": -0.15...","{""BC_SCORE"": 648, ""UTIL_LINEA_TDC"": 0.24, ""SAL..."
...,...,...,...,...,...,...
195,USR-00134,2025-06-25 23:44:15,2025-06-25 23:44:15+00:00,0.508,"{""BC_SCORE"": 0.253539, ""UTIL_LINEA_TDC"": -0.20...","{""BC_SCORE"": 640, ""UTIL_LINEA_TDC"": 0.34, ""SAL..."
196,USR-00062,2025-06-26 00:25:15,2025-06-26 00:25:15+00:00,0.498,"{""BC_SCORE"": 0.325863, ""UTIL_LINEA_TDC"": -0.03...","{""BC_SCORE"": 679, ""UTIL_LINEA_TDC"": 0.81, ""SAL..."
197,USR-00123,2025-06-26 00:37:15,2025-06-26 00:37:15+00:00,0.561,"{""BC_SCORE"": -0.039909, ""UTIL_LINEA_TDC"": 0.07...","{""BC_SCORE"": 720, ""UTIL_LINEA_TDC"": 0.91, ""SAL..."
198,USR-00164,2025-06-26 01:10:15,2025-06-26 01:10:15+00:00,0.492,"{""BC_SCORE"": 0.075587, ""UTIL_LINEA_TDC"": -0.14...","{""BC_SCORE"": 671, ""UTIL_LINEA_TDC"": 0.05, ""SAL..."


# Modelo

Usaremos el modelo GPT4.1-nano de OpenAI.

In [ ]:
from google.colab import userdata
# ------------------ CONFIGURACIÓN OPENAI ------------------
openai.api_key = userdata.get("OPENAI_API_KEY")

# ------------------ CARGAR DATOS ------------------
file_path = 'snowflake_multiple_simulated.csv'
data = pd.read_csv(file_path)
data['CALCULATION_DATE'] = pd.to_datetime(data['CALCULATION_DATE'], errors='coerce')
data = data.sort_values('CALCULATION_DATE').drop_duplicates(subset=['RPP_USER_ID'], keep='last')
data['FECHA_SOLICITUD'] = pd.to_datetime(data['FECHA_SOLICITUD'], errors='coerce', utc=True)
data = data.dropna(subset=['FECHA_SOLICITUD'])

# ------------------ CARGAR DEFINICIONES ------------------
file_path_desc = 'definiciones_sim.csv'
descriptions = pd.read_csv(file_path_desc, encoding='ISO-8859-1').dropna()
description_dict = pd.Series(descriptions.DESCRIPTION.values, index=descriptions.VARIABLE).to_dict()

In [ ]:
# ------------------ CARGAR DATOS ------------------
file_path = 'snowflake_multiple_simulated.csv'
data = pd.read_csv(file_path)
data['CALCULATION_DATE'] = pd.to_datetime(data['CALCULATION_DATE'], errors='coerce')
data = data.sort_values('CALCULATION_DATE').drop_duplicates(subset=['RPP_USER_ID'], keep='last')
data['FECHA_SOLICITUD'] = pd.to_datetime(data['FECHA_SOLICITUD'], errors='coerce', utc=True)
data = data.dropna(subset=['FECHA_SOLICITUD'])

# ------------------ CARGAR DEFINICIONES ------------------
file_path_desc = 'definiciones_sim.csv'
descriptions = pd.read_csv(file_path_desc, encoding='ISO-8859-1').dropna()
description_dict = pd.Series(descriptions.DESCRIPTION.values, index=descriptions.VARIABLE).to_dict()

## Distintos tipos de prompt

Las siguientes funciones definen tres tipos de prompt:

1. Uno simple
2. Uno usando la estructura Chain-of-Thought
3. Uno usando la aproximación Tree-of-Thought

| Tipo de Prompt         | Nombre corto | Estilo                          | Objetivo principal                                             | Estructura esperada                   | Ventajas                                                      | Cómo se construyó                                                                 |
|------------------------|--------------|----------------------------------|----------------------------------------------------------------|----------------------------------------|----------------------------------------------------------------|------------------------------------------------------------------------------------|
| **Simple**             | `Simple`     | Directo                          | Pedir una explicación resumida con base en factores            | Un solo párrafo o lista breve          | Rápido, directo, fácil de leer                               | Se concatena el texto de la predicción con una lista formateada de factores clave |
| **Chain-of-Thought**   | `CoT`        | Paso a paso secuencial           | Razonar cada factor individual antes de concluir               | Serie de pasos + conclusión final       | Favorece claridad lógica y trazabilidad                      | Se usa el mismo formato de lista pero con una instrucción inicial para analizar paso a paso |
| **Tree-of-Thought**    | `ToT`        | Razonamiento estructurado jerárquico | Separar factores positivos y negativos + reflexionar + resumir | Favorables/Desfavorables + resumen     | Máxima interpretabilidad, ideal para explicaciones ejecutivas | Se estructura el prompt en secciones: tarea, datos, factores e instrucciones explícitas estilo árbol |


In [ ]:
def probar_explicacion(user_id):
    import openai
    import pandas as pd
    import json

    # Cargar datos
    data = pd.read_csv("snowflake_multiple_simulated.csv")
    data["CALCULATION_DATE"] = pd.to_datetime(data["CALCULATION_DATE"])
    data["FECHA_SOLICITUD"] = pd.to_datetime(data["FECHA_SOLICITUD"], utc=True)

    description_dict = pd.read_csv("definiciones_sim.csv", encoding="ISO-8859-1") \
                        .set_index("VARIABLE")["DESCRIPTION"].to_dict()

    # Cliente OpenAI
    client = openai.OpenAI(api_key=openai.api_key)  # ← reemplaza con tu API key

    # Fila seleccionada
    row = data[data["RPP_USER_ID"] == user_id].iloc[0]
    score = row["SCORE"]
    pred = "El cliente fue aceptado." if score <= 0.172 else "El cliente fue rechazado."

    shap = json.loads(row["SHAP_JSON"])
    vals = json.loads(row["VAR_JSON"])

    # Crear DataFrame con top 5
    df = pd.DataFrame({
        "Features": list(shap.keys()),
        "SHAP_Values": list(shap.values()),
        "Raw_Values": [vals[k] for k in shap.keys()]
    })
    df["Abs"] = df["SHAP_Values"].abs()
    df["Description"] = df["Features"].map(description_dict)
    df = df.sort_values("Abs", ascending=False).head(5)

    # Prompts
    def build_simple(pred, df):
        facts = "\n".join(f"- {r.Description}: valor {r.Raw_Values}, SHAP {r.SHAP_Values:+.3f}" for _, r in df.iterrows())
        return f"{pred}\n\nFactores clave:\n{facts}\n\nResume por qué se tomó esta decisión."

    def build_cot(pred, df):
        facts = "\n".join(f"- {r.Description}: valor {r.Raw_Values}, SHAP {r.SHAP_Values:+.3f}" for _, r in df.iterrows())
        return f"{pred}\n\nAnaliza paso a paso cada uno de los factores y luego da una conclusión:\n{facts}"

    def build_tot(pred, df):
        facts = "\n".join(
        f"🔹 {r.Description}: valor {r.Raw_Values}, SHAP {r.SHAP_Values:+.3f}"
        for _, r in df.iterrows()
    )

        return f"""📌 [TAREA]
Eres analista de riesgo. Explica PASO A PASO por qué el cliente fue {'✅ ACEPTADO' if 'aceptado' in pred else '❌ RECHAZADO'}.

📊 [DATOS DEL CASO]
{pred}

📌 [FACTORES CLAVE]
{facts}

🧠 [INSTRUCCIONES ToT]
1️⃣ Enumera los factores que 📈 favorecen y los que 📉 afectan negativamente la decisión.
2️⃣ Escribe una reflexión 🗒️ breve por cada uno.
3️⃣ Finaliza con un ✍️ resumen ejecutivo (2–3 líneas) sin jerga técnica.
4️⃣ 🎯 Usa emojis adecuados (📈, 📉, ✅, ❌, 💡, etc.) en los encabezados y reflexiones para mejorar la claridad visual.

### 🧩 Razonamiento
- (📈 Favorable) …
- (📉 Desfavorable) …

### 📋 Resumen final
…"""


    prompts = {
        "Simple": build_simple(pred, df),
        "CoT": build_cot(pred, df),
        "ToT": build_tot(pred, df)
    }

    for name, prompt in prompts.items():
        print(f"\n\n=== {name.upper()} ===\n")
        response = client.chat.completions.create(
            model="gpt-4.1-nano",
            messages=[
                {"role": "system", "content": "Eres analista de riesgo de crédito."},
                {"role": "user", "content": prompt}
            ],
            temperature=0.7,
            max_tokens=500
        )
        print(response.choices[0].message.content)




In [ ]:

import pandas as pd
# The 'row' variable contains the information for the specific user.
# We need to extract the JSON strings for SHAP and VAR and then load them.

user_id = "USR-00042"
row = data[data["RPP_USER_ID"] == user_id].iloc[0]

shap_data = json.loads(row["SHAP_JSON"])
var_data = json.loads(row["VAR_JSON"])

# Combine the extracted data into a dictionary
user_characteristics = {
    "user_id": user_id,
    "SCORE": row["SCORE"],
    "CALCULATION_DATE": row["CALCULATION_DATE"],
    "FECHA_SOLICITUD": row["FECHA_SOLICITUD"],
    "SHAP_Values": shap_data,
    "Raw_Values": var_data
}

# Convert the dictionary into a pandas DataFrame for table display
user_df = pd.DataFrame([user_characteristics])

# Expand the JSON columns for better readability in the table if needed,
# but for a simple display, we can just show the dictionaries.
# If you want to expand them into separate columns, you would do:
shap_df = pd.DataFrame([shap_data])
var_df = pd.DataFrame([var_data])
user_df = pd.concat([user_df[['user_id', 'SCORE', 'CALCULATION_DATE', 'FECHA_SOLICITUD']], shap_df, var_df], axis=1)


user_df


,user_id,SCORE,CALCULATION_DATE,FECHA_SOLICITUD,BC_SCORE,UTIL_LINEA_TDC,SALDO_VIGENTE,ANTIG_EDO_CTA,TIEMPO_EMPLEO,INGRESOS_MENSUALES,BC_SCORE,UTIL_LINEA_TDC,SALDO_VIGENTE,ANTIG_EDO_CTA,TIEMPO_EMPLEO,INGRESOS_MENSUALES
0,USR-00042,0.607,2025-06-23 23:49:15,2025-06-23 23:49:15+00:00,-0.081397,-0.109931,-0.154997,-0.034278,-0.006264,-0.279654,693,0.24,125696.0,55,29,17402.34


In [ ]:
probar_explicacion("USR-00042")  # reemplaza con un ID válido del CSV



=== SIMPLE ===

La decisión de rechazo del cliente se basó en un análisis de riesgo que consideró varios factores clave. Aunque los ingresos mensuales declarados son relativamente altos (17,402.34), su impacto en el modelo fue negativo (-0.280 SHAP), lo que indica que no contribuyó a una evaluación favorable del riesgo. Además, el saldo vigente en productos activos es elevado (125,696.0), con un impacto negativo (-0.155 SHAP), sugiriendo una posible carga de deuda elevada. La utilización de la línea de tarjeta de crédito (24%) también mostró un impacto negativo (-0.110 SHAP), reflejando un nivel de utilización que puede considerarse moderado a alto. La calificación en Buró de Crédito (693) tuvo un impacto negativo menor (-0.081 SHAP), indicando una situación crediticia que no favorece la aprobación. Finalmente, la antigüedad del estado de cuenta (55 meses) tuvo un impacto mínimo (-0.034 SHAP), pero contribuyó a la decisión general. En conjunto, estos factores sugieren un perfil de ri

# Reflexiones finales

Al comparar los tres estilos de explicación —Simple, Chain-of-Thought (CoT) y Tree-of-Thought (ToT)— notamos diferencias clave en profundidad, estructura y utilidad interpretativa 🧠. El enfoque Simple nos da una descripción directa del resultado, pero se siente más mecánico y menos pedagógico; es útil para entender “qué” ocurrió, pero no tanto el “por qué”. En contraste, el estilo CoT nos guía paso a paso 🪜 por cada variable, permitiendo razonar de forma secuencial sobre su impacto individual y entender cómo se combinan en la decisión, lo cual favorece la trazabilidad y la transparencia. Finalmente, el estilo ToT sobresale por su organización jerárquica 🌳: separa claramente los factores favorables (📈) y desfavorables (📉), incluye reflexiones por dimensión y ofrece un resumen ejecutivo claro 📋. Este último resulta especialmente útil cuando queremos comunicar resultados a usuarios no técnicos o incorporar explicaciones en productos con alta sensibilidad (como clientes financieros).